In [2]:
#imports
using Pkg; for p in ("Knet","IterTools"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Statistics: mean
using IterTools: ncycle
using WordTokenizers
using Test
using Knet
using Random

In [3]:
train_file = "/home/burak/nn4nlp-code/data/classes/train.txt"
test_file = "/home/burak/nn4nlp-code/data/classes/test.txt"

"/home/burak/nn4nlp-code/data/classes/test.txt"

In [4]:
wdict = Dict()
w2i(x)= get!(wdict,x,1+length(wdict))
tdict = Dict()
t2i(x) = get!(tdict,x,1+length(tdict))


t2i (generic function with 1 method)

In [5]:
UNK = w2i("<unk>")

1

In [6]:
wdict

Dict{Any,Any} with 1 entry:
  "<unk>" => 1

In [7]:
function read_dataset(filename)
    sent_arr = []
  open(filename, "r") do f
    for line in eachline(f)
        line = lowercase(line)
        tag,words = split(strip(line)," ||| ")
        sent = split(words," ")
        sent = [w2i.(sent)]
        push!(sent_arr,[sent,t2i(tag)])
        end
    end
    return sent_arr
end


read_dataset (generic function with 1 method)

In [8]:
train = [(r(train_file))]
nwords = length(wdict)
test = [(r(test_file))]
S = w2i("<s>")
@test nwords == length(wdict)


UndefVarError: UndefVarError: r not defined

In [9]:

# Read in the data
train = [read_dataset(train_file)]
w2i(x)= get!(wdict,x,1)
dev = [read_dataset(test_file)]
nwords = length(wdict)
ntags = length(tdict)

5

In [10]:
train =reshape(train[1],(length(train[1])))
dev = reshape(dev[1],(length(dev[1])))

2210-element Array{Any,1}:
 Any[Array{Int64,1}[[1796, 72, 1, 449]], 3]                                                                                       
 Any[Array{Int64,1}[[234, 81, 514, 168, 6, 872, 6, 2, 598, 6  …  147, 27, 2967, 4, 20, 146, 1269, 6, 4297, 34]], 1]               
 Any[Array{Int64,1}[[2209, 227, 1691, 1026, 27, 111, 2287, 310, 16, 10  …  3127, 3128, 16, 133, 945, 317, 1034, 168, 286, 34]], 2]
 Any[Array{Int64,1}[[2, 267, 641, 190, 280, 823, 314, 2, 10169, 5403  …  396, 88, 10226, 2, 8132, 5654, 38, 2, 2738, 34]], 3]     
 Any[Array{Int64,1}[[1027, 16, 1026, 2699, 38, 1203, 15, 5905, 34]], 2]                                                           
 Any[Array{Int64,1}[[1617, 861, 1059, 670, 485, 246, 68, 12452, 1, 16, 2, 653, 6, 1528, 4, 12219, 94, 146, 4621, 34]], 1]         
 Any[Array{Int64,1}[[2379, 558, 119, 20, 5729, 138, 16, 1, 172, 2  …  133, 10, 42, 1670, 81, 507, 6, 3487, 133, 34]], 1]          
 Any[Array{Int64,1}[[72, 17, 2101, 3146, 133, 581, 34]],

In [11]:
@test nwords == 18648
@test ntags == 5
@test length(train) == 8544
@test length(dev)== 2210

Test Passed

In [12]:
EMB_SIZE = 64
HID_SIZE = 64
W_emb = param(nwords,EMB_SIZE)

18648×64 Param{Array{Float32,2}}:
  0.00571242   -0.00466897   -0.000777082  …   0.00249087    0.00702455 
 -0.00047844   -0.00805948    0.00613492      -0.0067205     0.000879379
  0.00437906   -0.00476924    0.00544929       0.00734617    0.00970492 
  0.0058231    -0.000718723  -0.00283531      -0.00561079    5.20598e-5 
 -0.00548728   -0.00178591    0.00346871      -0.000352573  -0.00514146 
 -0.00816576    0.00866473   -0.00460712   …  -0.00142837   -3.78005e-5 
 -0.00244249   -0.00517956    0.00286727       0.00843817    0.00526111 
  0.00923522    0.0103306     0.00526529       0.00974129    0.00360895 
  0.00625398   -0.00416582    0.00909361      -0.00633676   -0.00678761 
 -0.000757157  -0.00936348   -0.0101423        0.00193349   -0.00441899 
 -0.00310765    0.0097769     0.00667075   …  -0.00295127    0.00429946 
 -0.00313611    0.00349517    0.00722661      -0.00489251    0.0100096  
 -0.00386809   -0.00499072    0.000974707      0.00635774    0.00627471 
  ⋮              

In [13]:
sum(W_emb[1,:])

-0.0224399f0

In [14]:
length(W_emb[[1,2,3],:])

192

In [15]:
@doc param

```
param(array; atype)
param(dims...; init, atype)
param0(dims...; atype)
```

The first form returns `Param(atype(array))` where `atype=identity` is the default.

The second form Returns a randomly initialized `Param(atype(init(dims...)))`. By default, `init` is `xavier` and `atype` is `KnetArray{Float32}` if `gpu() >= 0`, `Array{Float32}` otherwise. 

The third form `param0` is an alias for `param(dims...; init=zeros)`.


In [20]:
fwdLSTM = RNN(EMB_SIZE ,HID_SIZE,numLayers= 1 ,rnnType =:tanh)
bwdLSTM = RNN(EMB_SIZE ,HID_SIZE,numLayers= 1 ,rnnType =:tanh)
W_sm = param(ntags,2*HID_SIZE)
b_sm = param(ntags)

5-element Param{Array{Float32,1}}:
 -0.14857644
 -0.34483647
  0.39617085
 -0.32704997
  0.23111926

In [17]:
y =rnn(WORDS_LOOKUP[S,:])

UndefVarError: UndefVarError: WORDS_LOOKUP not defined

In [18]:
@doc RNN

```
rnn = RNN(inputSize, hiddenSize; opts...)
rnn(x; batchSizes) => y
rnn.h, rnn.c  # hidden and cell states
```

`RNN` returns a callable RNN object `rnn`. Given a minibatch of sequences `x`, `rnn(x)` returns `y`, the hidden states of the final layer for each time step. `rnn.h` and `rnn.c` fields can be used to set the initial hidden states and read the final hidden states of all layers.  Note that the final time step of `y` always contains the final hidden state of the last layer, equivalent to `rnn.h` for a single layer network.

**Dimensions:** The input `x` can be 1, 2, or 3 dimensional and `y` will have the same number of dimensions as `x`. size(x)=(X,[B,T]) and size(y)=(H/2H,[B,T]) where X is inputSize, B is batchSize, T is seqLength, H is hiddenSize, 2H is for bidirectional RNNs. By default a 1-D `x` represents a single instance for a single time step, a 2-D `x` represents a single minibatch for a single time step, and a 3-D `x` represents a sequence of identically sized minibatches for multiple time steps. The output `y` gives the hidden state (of the final layer for multi-layer RNNs) for each time step. The fields `rnn.h` and `rnn.c` represent the hidden states of all layers in a single time step and have size (H,B,L/2L) where L is numLayers and 2L is for bidirectional RNNs.

**batchSizes:** If `batchSizes=nothing` (default), all sequences in a minibatch are assumed to be the same length. If `batchSizes` is an array of (non-increasing) integers, it gives us the batch size for each time step (allowing different sequences in the minibatch to have different lengths). In this case `x` will typically be 2-D with the second dimension representing variable size batches for time steps. If `batchSizes` is used, `sum(batchSizes)` should equal `length(x) ÷ size(x,1)`. When the batch size is different in every time step, hidden states will have size (H,B,L/2L) where B is always the size of the first (largest) minibatch.

**Hidden states:** The hidden and cell states are kept in `rnn.h` and `rnn.c` fields (the cell state is only used by LSTM). They can be initialized during construction using the `h` and `c` keyword arguments, or modified later by direct assignment. Valid values are `nothing` (default), `0`, or an array of the right type and size possibly wrapped in a `Param`. If the value is `nothing` the initial state is assumed to be zero and the final state is discarded keeping the value `nothing`. If the value is `0` the initial state is assumed to be zero and `0` is replaced by the final state on return. If the value is a valid state, it is used as the initial state and is replaced by the final state on return.

In a differentiation context the returned final hidden states will be wrapped in `Result` types. This is necessary if the same RNN object is to be called multiple times in a single iteration. Between iterations (i.e. after diff/update) the hidden states need to be unboxed with e.g. `rnn.h = value(rnn.h)` to prevent spurious dependencies. This happens automatically during the backward pass for GPU RNNs but needs to be done manually for CPU RNNs. See the [CharLM Tutorial](https://github.com/denizyuret/Knet.jl/blob/master/tutorial/80.charlm.ipynb) for an example.

**Keyword arguments for RNN:**

  * `h=nothing`: Initial hidden state.
  * `c=nothing`: Initial cell state.
  * `rnnType=:lstm` Type of RNN: One of :relu, :tanh, :lstm, :gru.
  * `numLayers=1`: Number of RNN layers.
  * `bidirectional=false`: Create a bidirectional RNN if `true`.
  * `dropout=0`: Dropout probability. Applied to input and between layers.
  * `skipInput=false`: Do not multiply the input with a matrix if `true`.
  * `dataType=Float32`: Data type to use for weights.
  * `algo=0`: Algorithm to use, see CUDNN docs for details.
  * `seed=0`: Random number seed for dropout. Uses `time()` if 0.
  * `winit=xavier`: Weight initialization method for matrices.
  * `binit=zeros`: Weight initialization method for bias vectors.
  * `finit=ones`: Weight initialization method for the bias of forget gates.
  * `usegpu=(gpu()>=0)`: GPU used by default if one exists.

**Formulas:** RNNs compute the output h[t] for a given iteration from the recurrent input h[t-1] and the previous layer input x[t] given matrices W, R and biases bW, bR from the following equations:

`:relu` and `:tanh`: Single gate RNN with activation function f:

```
h[t] = f(W * x[t] .+ R * h[t-1] .+ bW .+ bR)
```

`:gru`: Gated recurrent unit:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
r[t] = sigm(Wr * x[t] .+ Rr * h[t-1] .+ bWr .+ bRr) # reset gate
n[t] = tanh(Wn * x[t] .+ r[t] .* (Rn * h[t-1] .+ bRn) .+ bWn) # new gate
h[t] = (1 - i[t]) .* n[t] .+ i[t] .* h[t-1]
```

`:lstm`: Long short term memory unit with no peephole connections:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
f[t] = sigm(Wf * x[t] .+ Rf * h[t-1] .+ bWf .+ bRf) # forget gate
o[t] = sigm(Wo * x[t] .+ Ro * h[t-1] .+ bWo .+ bRo) # output gate
n[t] = tanh(Wn * x[t] .+ Rn * h[t-1] .+ bWn .+ bRn) # new gate
c[t] = f[t] .* c[t-1] .+ i[t] .* n[t]               # cell output
h[t] = o[t] .* tanh(c[t])
```


In [21]:
function calc_scores(words)
    word_embs = W_emb[words,:]
    fwdLSTM.h= 0 
    fwdLSTM.c = 0
    fwd_embs = Param(word_embs)
    bwdLSTM.h=0
    bwdLSTM.c =0
    bwd_embs = Param(reverse(word_embs,dims=1))
    embs = cat(fwd_embs[end,:],bwd_embs[end,:],dims=1)
    return W_sm*embs+b_sm
    


end

calc_scores (generic function with 1 method)

In [23]:

sum(calc_scores([2,3,4,5]))

-0.18465272f0

In [59]:
@doc concatenate

No documentation found.

Binding `concatenate` does not exist.


In [55]:

@doc reverse

```
reverse(v [, start=1 [, stop=length(v) ]] )
```

Return a copy of `v` reversed from start to stop.  See also [`Iterators.reverse`](@ref) for reverse-order iteration without making a copy.

# Examples

```jldoctest
julia> A = Vector(1:5)
5-element Array{Int64,1}:
 1
 2
 3
 4
 5

julia> reverse(A)
5-element Array{Int64,1}:
 5
 4
 3
 2
 1

julia> reverse(A, 1, 4)
5-element Array{Int64,1}:
 4
 3
 2
 1
 5

julia> reverse(A, 3, 5)
5-element Array{Int64,1}:
 1
 2
 5
 4
 3
```

```
reverse(A; dims::Integer)
```

Reverse `A` in dimension `dims`.

# Examples

```jldoctest
julia> b = [1 2; 3 4]
2×2 Array{Int64,2}:
 1  2
 3  4

julia> reverse(b, dims=2)
2×2 Array{Int64,2}:
 2  1
 4  3
```

```
reverse(s::AbstractString) -> AbstractString
```

Reverses a string. Technically, this function reverses the codepoints in a string and its main utility is for reversed-order string processing, especially for reversed regular-expression searches. See also [`reverseind`](@ref) to convert indices in `s` to indices in `reverse(s)` and vice-versa, and `graphemes` from module `Unicode` to operate on user-visible "characters" (graphemes) rather than codepoints. See also [`Iterators.reverse`](@ref) for reverse-order iteration without making a copy. Custom string types must implement the `reverse` function themselves and should typically return a string with the same type and encoding. If they return a string with a different encoding, they must also override `reverseind` for that string type to satisfy `s[reverseind(s,i)] == reverse(s)[i]`.

# Examples

```jldoctest
julia> reverse("JuliaLang")
"gnaLailuJ"

julia> reverse("ax̂e") # combining characters can lead to surprising results
"êxa"

julia> using Unicode

julia> join(reverse(collect(graphemes("ax̂e")))) # reverses graphemes
"ex̂a"
```


In [ ]:
train_order = (1:length(train))

In [ ]:
length(train_order)

In [ ]:
print("startup time: %r" % (time() - start))


In [ ]:
i = all_time = dev_time = all_tagged = this_words = this_loss = 0


In [ ]:
for ITER in 1:100
    print(ITER)
end

In [ ]:
shuffle(train_order)

In [ ]:
test[2]

In [ ]:
exp(2)

In [ ]:
a =5/2
print("a equals"*string(a)*"asd")

In [ ]:
#print("startup time: %r" % (time.time() - start))
# Perform training
#start = time.time()
i = all_time = dev_time = all_tagged = this_words = this_loss = 0
for ITER in 1:100
    shuffle(train_order)
    for sid in train_order
        i += 1
        if i % 1 == 0
            #trainer.status()
            print(this_loss / this_words)
            all_tagged += this_words
            this_loss = this_words = 0
           # all_time = time.time() - start
        end
        if i % 2 == 0
            #dev_start = time.time()
            dev_loss = dev_words = 0
            for sent in test
                loss_exp, mb_words = calc_lm_loss(sent)
                dev_loss += loss_exp
                dev_words += mb_words
            end
           # dev_time += time.time() - dev_start
            #train_time = time.time() - start - dev_time
            nl = dev_loss/dev_words
            ppl = exp(dev_loss/dev_words)
            print("NLL: "*nl*" ppl : "*ppl)
        # train on the minibatch
        loss_exp, mb_words = calc_lm_loss(train[sid])
        this_loss += loss_exp
        this_words += mb_words
        loss_exp.backward()
        trainer.update()
        end
    end
    print("epoch %r finished" % ITER)
    trainer.update_epoch(1.0)
end